<img src='assets/svhn_dcgan.png' width=80% />

# Deep Convolutional GANs

training DCGAN on the [Street View House Numbers](http://ufldl.stanford.edu/housenumbers/) (SVHN) dataset.

In [1]:
# import all the packages:
import matplotlib.pyplot as plt
import torch
import numpy as np
import pickle as pkl

%matplotlib inline


In [ ]:
from torchvision import datasets
from torchvision import transforms


#convert to pytorch tensors
